In [1]:
"""
toolkit
Usage:
python3 *.py
https://www.gymlibrary.dev/environments/classic_control/cart_pole/

# toolkit_for_exploration_in_RL_
this is supporting toolking for the paper : Experiments Focused on Exploration in Deep Reinforcement Learning with doi: DOI: 10.1109/ISMSIT52890.2021.9604690


This toolkit is create to ease experiments with RL-exploration
options:
1. Optional test described in the paper
1.1. no restictricted exploration
1.2. randomized explorarion
1.3. artificial restrictions
1.4 lead by teacher explorations
2. options for custom size ANN
3. options for custom sized buffer replay
4. options for build in plots (visualizations of results)
5. option for choosing size of training episodes and test episodes
6. option for overestemation checking (described in the paper)

BONUS: REALtime checking how advantage changes weights of the ANNs during training

Do please e-mail at: kaloev_92@mail.ru 

If this toolkit may be usefull for you, used!!!!
Citation of the paper will be much apriciated, 
Have a nice day dear reader. 


this is my ocena code updates ipynb
===========
IMPORTANT FOR VERSIONS
keras , tf version - 2.6.0 ; do change into proper tensor version (shapes for 2.11 or v3)
gym version 0.15.7
"""
import sys
import gym
import tensorflow as tf
import numpy as np
import os
from tensorflow import keras
import matplotlib.pyplot as plt
from collections import deque
import time
import random

print(tf. __version__) 
print(keras.__version__)
print("User Current Version:-", sys.version)
print(gym.__version__)

2022-12-23 12:27:31.347651: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-23 12:27:31.347678: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.6.0
2.6.0
User Current Version:- 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
0.15.7


In [2]:
RANDOM_SEED = 6
tf.random.set_seed(RANDOM_SEED)

env = gym.make('CartPole-v1')
#env.seed(RANDOM_SEED)
#np.random.seed(RANDOM_SEED)

print("Action Space: {}".format(env.action_space))
print("State space: {}".format(env.observation_space))

Action Space: Discrete(2)
State space: Box(4,)


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
# An episode a full game
com=input("to enter into custom setings press c or 1: ")
if com=="c" or com=="1":
    print("custtom settings menu: ")
    train_episodes = int(input("chose how many episode is whole test: "))
    ctr_ep= int(input("chose how many episode are spend in training "))
    buffer=int(input("buffer size, warning this will efect after how many action the ANN is updated: "))
    custom_deept_layers_actor= int( input("custom size of actor deppt, warning, choosing incorct dept may cause overfitting: "))
    custom_deept_layers_critic= int( input("custom size of critic deppt, warning, choosing incorct dept may cause overfitting: "))
    
else:
    print("deffult settings used ")
    train_episodes = 90
    ctr_ep=60
    buffer=2
    custom_deept_layers_actor=2
    custom_deept_layers_critic=2

to enter into custom setings press c or 1:  0


deffult settings used 


In [4]:

def create_actor(state_shape, action_shape, func_,i=3):
    
    init = tf.keras.initializers.HeUniform()
    model = keras.Sequential()
    model.add(keras.layers.Dense(124, input_shape=state_shape, activation=tf.keras.layers.LeakyReLU(), kernel_initializer=init))
    for j in range(0, i):
        model.add(keras.layers.Dense(112, activation=tf.keras.layers.LeakyReLU(), kernel_initializer=init))
 
    model.add(keras.layers.Dense(action_shape, activation=func_ , kernel_initializer=init))
    
    model.summary()
    return model

In [5]:
def create_critic(state_shape, output_shape,i=4):
    
    init = tf.keras.initializers.HeUniform()
    model = keras.Sequential()
    model.add(keras.layers.Dense(124, input_shape=state_shape, activation=tf.keras.layers.LeakyReLU(), kernel_initializer=init))
    for j in range(0, i):
        model.add(keras.layers.Dense(112, activation=tf.keras.layers.LeakyReLU(), kernel_initializer=init))

    model.add(keras.layers.Dense(output_shape, activation='linear', kernel_initializer=init))
    
    return model

In [6]:
def create_actor2(state_shape, action_shape):
    
    init = tf.keras.initializers.HeUniform()
    model = keras.Sequential()
    model.add(keras.layers.Dense(124, input_shape=state_shape, activation=tf.keras.layers.LeakyReLU(), kernel_initializer=init))
    model.add(keras.layers.Dense(12, activation=tf.keras.layers.LeakyReLU(), kernel_initializer=init))
    model.add(keras.layers.Dense(action_shape, activation='softmax', kernel_initializer=init))
    
    return model

In [7]:
#ploting fature
def plts(rewards,name_fig,r="Rewards", e="Episodes"):
	plt.xlabel(e)
	plt.ylabel(r)
	plt.title(name_fig)
	plt.plot(rewards)
	#plt.plot(ep)
	#plt.legend(loc=0)
	plt.savefig(name_fig) 
	plt.show()      


#add to check name oof off actions
def savs(fl_name,rewards_list):
    ff=open(fl_name+".txt","w")
    ff.write(str(rewards_list))
    ff.close()

In [8]:
def main(ctr_l):
    actor_checkpoint_path = "training_actor/actor_cp.ckpt"
    critic_checkpoint_path = "training_critic/critic_cp.ckpt"
    actor_checkpoint_path2 = "training_actor/actor_cp.hdf5"
    critic_checkpoint_path2 = "training_critic/critic_cp.hdf5"


    
    actor_checkpoint_path_ = "training_actor_/actor_cp.ckpt"
    actor_checkpoint_path2_ = "training_actor_/actor_cp.hdf5"
    

    actor = create_actor(env.observation_space.shape, env.action_space.n, 'softmax', custom_deept_layers_actor)
    actor.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['accuracy'])

    actor_overest = create_actor(env.observation_space.shape, env.action_space.n, "linear" , custom_deept_layers_actor)
    actor_overest.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['accuracy'])

    critic = create_critic(env.observation_space.shape, 1 , custom_deept_layers_critic)
    critic.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['accuracy'])
    

    actor2 = create_actor2(env.observation_space.shape, env.action_space.n)
    actor2.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['accuracy'])
    #add teacher ACTOR HERE




    #saaves maybe
    if os.path.exists('training_actor'):
        actor.load_weights(actor_checkpoint_path)

        critic.load_weights(critic_checkpoint_path)
    
    if os.path.exists('training_actor_'):
        actor2.load_weights(actor_checkpoint_path_)
        print("LOADED")
    else:
        print("crash")
 

    #BUFFERS
    rewards_=[]
    rewards_2=[]
    obs_=[]
    advs_=[]
    acts_=[]
    long_advs_=[]
    tds_=[]
    acts_raw_=[]
    over_left_=[]
    over_rigght_=[]
    #BUFFERS

    for episode in range(train_episodes):
        total_training_rewards = 0
        action_c=0
        observation = env.reset()
        done = False
        while not done:
            if True:
                #DO NOT RENDER HERE IT . CRASH CODEOCEAN
                #env.render()
                pass

            # model dims are (batch, env.observation_space.n)
            # tensoers convers

            observation_reshaped = tf.convert_to_tensor(observation)
            observation_reshaped = tf.expand_dims(observation_reshaped,0)
            action_probs = actor.predict(observation_reshaped).flatten()
            action_probs2 = actor2.predict(observation_reshaped).flatten()

            
            
            #action add custom
            if ctr_l==0:
                action =(np.argmax(action_probs))
            elif ctr_l==2:
                if episode < ctr_ep:
                    if (observation[2] > .17):
                        action=1
                    else:
                        action=(np.argmax(action_probs))
                else:
                    action=(np.argmax(action_probs))
            elif ctr_l==1:
                if episode < ctr_l:
                    action = env.action_space.sample()
                else:
                    action=(np.argmax(action_probs))
            elif ctr_l==3:
                if episode < ctr_l:
                    action=(np.argmax(action_probs2))
                else:
                    action=(np.argmax(action_probs))
            else:
                print("CRASHED")
            

            next_observation, reward, done, info = env.step(action)
            next_observation_reshaped = tf.convert_to_tensor(next_observation)
            next_observation_reshaped = tf.expand_dims(next_observation_reshaped,0)
            

            #add size here for train eps_
            if (episode <ctr_ep):
                value_curr = np.asscalar(np.array(critic.predict(observation_reshaped)))
                value_next = np.asscalar(np.array(critic.predict(next_observation_reshaped)))

            # calculation of A= (r+V(s') - V (s))
                discount_factor = .7
                Temp_Diff = reward + (1 - done) * discount_factor * value_next
                advantage = Temp_Diff- value_curr
            
                advantage_reshaped = np.vstack([advantage])
                TD_target = np.vstack([Temp_Diff])

                #mid batch mormalisation traing for V*(s)
                critic.train_on_batch(observation_reshaped, TD_target)

                acts_raw_.append(action_probs)
                obs_.append(observation)
                acts_.append(action)
                advs_.append(advantage)
                tds_.append(Temp_Diff)
                
                if len(obs_)>buffer:

                    #apply gradiotion and train ANNS
                    actor.fit(np.array(obs_), np.array(acts_), sample_weight=np.array(advs_), epochs=10 ,verbose=0)
                    critic.fit(np.array(obs_), np.array(tds_), epochs=10 ,verbose=0)


                    #REAL TIME UPDATES features
                    for i in range(len(obs_)):
                        _saved_=obs_[i]
                        _saved_ = tf.convert_to_tensor(_saved_)
                        _saved_= tf.expand_dims(_saved_,0)
                        updated_=actor.predict(_saved_)
                        print("observation-f: ", np.round(obs_[i],3), " pre update acts: ", np.round(acts_raw_[i],3)," advantage used: ",np.round(advs_[i],2) , " post update acts: ", np.round(updated_[0],3))
                    
                    
                    obs_.clear()
                    acts_.clear()
                    advs_.clear()
                    long_advs_.clear()
                    tds_.clear()
                    acts_raw_.clear()
            
            observation = next_observation
            total_training_rewards += reward
            action_c+=1

            if episode > ctr_ep:
                
                #OVERESTEAMTION OBSERVATIONS FEATIRES
                over_left=(actor_overest.predict(observation_reshaped).flatten())[0]
                over_rigght=(actor_overest.predict(observation_reshaped).flatten())[1]

                over_left_.append(over_left)
                over_rigght_.append(over_rigght)

            if done:
                
                print ( "ep:",episode ,"act num:", action_c  ,"r:", np.round(reward, 2), "big r: ",total_training_rewards," adv: ",np.round(advantage, 2) ,"act:", np.round(action,2), "s-facor: ", observation, "act-space ", action )
                
                rewards_.append(total_training_rewards)
                if episode >ctr_ep:

                    actor_overest.set_weights(actor.get_weights())
                    rewards_2.append(total_training_rewards)
                    
                    
 
            


    env.close()
    if ctr_l==0:
        flag_="no restriction"
    elif ctr_l ==1:
        flag_="randomized moves"
    elif ctr_l ==2:
        flag_="restriction"
    elif ctr_l ==3:
        flag_="leaded by teacher"

    plts(rewards_,"Rewards per episode "+str(flag_))
    plts(rewards_2,"Rewards per episode_post chosing moves with "+str(flag_))

    plts(over_left_,"Overestemation left "+str(flag_),"overestemation", "actions")
    plts(over_rigght_,"Overestemation right "+str(flag_), "overestemations", "actions")
    #add plts for overestemations___

    savs("full_",rewards_)
    savs("post_tr",rewards_2)


In [9]:
if __name__ == '__main__':
    com=int(input("enter test rotine  \n 0 - no restriction run \n 1 - randomized exploration \n 2 - run with restrictions \n 3 - expl lead by teacher ANN : "))
    main(com)
    #main(1)
    #main(2)
    #main(3)
    #add for all tests

enter test rotine  
 0 - no restriction run 
 1 - randomized exploration 
 2 - run with restrictions 
 3 - expl lead by teacher ANN :  3


2022-12-23 12:27:38.621601: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-23 12:27:38.621628: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-23 12:27:38.621645: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (636b3d74423a): /proc/driver/nvidia/version does not exist
2022-12-23 12:27:38.622090: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` a

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 124)               620       
_________________________________________________________________
dense_1 (Dense)              (None, 112)               14000     
_________________________________________________________________
dense_2 (Dense)              (None, 112)               12656     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 226       
Total params: 27,502
Trainable params: 27,502
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 124)               620       
______________________________

2022-12-23 12:27:38.912203: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:109: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:110: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/opt/conda/lib/python3.7/site-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


observation-f:  [ 0.027  0.01  -0.013  0.003]  pre update acts:  [0.495 0.505]  advantage used:  0.97  post update acts:  [0.342 0.658]
observation-f:  [ 0.028  0.206 -0.013 -0.294]  pre update acts:  [0.591 0.409]  advantage used:  0.94  post update acts:  [0.971 0.029]
observation-f:  [ 0.032  0.011 -0.019 -0.005]  pre update acts:  [0.496 0.504]  advantage used:  1.17  post update acts:  [0.351 0.649]
observation-f:  [ 0.032  0.206 -0.019 -0.304]  pre update acts:  [0.974 0.026]  advantage used:  -0.21  post update acts:  [0.986 0.014]
observation-f:  [ 0.036  0.011 -0.025 -0.017]  pre update acts:  [0.375 0.625]  advantage used:  1.52  post update acts:  [0.144 0.856]
observation-f:  [ 0.036  0.207 -0.026 -0.318]  pre update acts:  [0.977 0.023]  advantage used:  -0.09  post update acts:  [0.988 0.012]
observation-f:  [ 0.04   0.012 -0.032 -0.034]  pre update acts:  [0.16 0.84]  advantage used:  0.93  post update acts:  [0.074 0.926]
observation-f:  [ 0.041  0.208 -0.033 -0.336]  p

KeyboardInterrupt: 